### All necessary imports

In [24]:
import urllib.request
from urllib.request import urlopen
import os
import glob
import shutil
import csv
import pprint
import pandas as pd
import numpy as np
from zipfile import ZipFile
from urllib.request import urlopen
from tempfile import NamedTemporaryFile
from shutil import unpack_archive
from io import BytesIO

### Downloading the files in zip

In [26]:
# Unzip and downloading the files in the folder

def openUnzip(url):
    with urlopen(url) as zipresp:
        with ZipFile(BytesIO(zipresp.read())) as zfile:
            zfile.extractall('C:/Users/madhu/Documents/NEU/Fall 2017/ADS Class/Assignment_1/Assign 1/Part_2')

In [ ]:

file_obj = open("C:/Users/madhu/Documents/NEU/Fall 2017/ADS Class/Assignment_1/Assign 1/Part_2/year.txt","r" )
year = file_obj.readline().rstrip()
zipurl = 'http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/'
generalUrl = "http://www.sec.gov/dera/data/PublicEDGAR-log-file-data/"
months = ['01',"02","03","04","05","06","07","08","09","10","11","12"]
for m in range(len(months)):
    if(str(months[m]) is "01" or str(months[m]) is "02" or str(months[m]) is "03"):
        urlAppended = zipurl + str(year) +"/Qtr1/"+"log"+str(year)+str(months[m])+"01.zip"
        print(urlAppended)
        openUnzip(urlAppended)
    if(str(months[m]) is "04" or str(months[m]) is "05" or str(months[m]) is "06"):
        urlAppended = zipurl + str(year) +"/Qtr2/"+"log"+str(year)+str(months[m])+"01.zip"
        print(urlAppended)
        openUnzip(urlAppended)
    if(str(months[m]) is "07" or str(months[m]) is "08" or str(months[m]) is "09"):
        urlAppended = zipurl + str(year) +"/Qtr3/"+"log"+str(year)+str(months[m])+"01.zip"
        print(urlAppended)
        openUnzip(urlAppended)
    if(str(months[m]) is "10" or str(months[m]) is "11" or str(months[m]) is "12"):
        urlAppended = zipurl + str(year) +"/Qtr4/"+"log"+str(year)+str(months[m])+"01.zip"
        print(urlAppended)
        openUnzip(urlAppended)

http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/2016/Qtr1/log20160101.zip
http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/2016/Qtr1/log20160201.zip
http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/2016/Qtr1/log20160301.zip
http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/2016/Qtr2/log20160401.zip
http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/2016/Qtr2/log20160501.zip
http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/2016/Qtr2/log20160601.zip
http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/2016/Qtr3/log20160701.zip
http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/2016/Qtr3/log20160801.zip
http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/2016/Qtr3/log20160901.zip
http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/2016/Qtr4/log20161001.zip
http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/2016/Qtr4/log20161101.zip


In [25]:
# year ='2013'
# from urllib.request import urlopen
# from tempfile import NamedTemporaryFile
# from shutil import unpack_archive
# zipurl = 'http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/2003/Qtr1/log20030201.zip'
# zipresp = urlopen(zipurl)
# tfile = NamedTemporaryFile()
# tfile.write(zipresp.read())
# tfile.seek(0)
# unpack_archive(tfile.name,path, format = 'zip')
# zipresp.close()
# tfile.close()

### Reading the CSV in a dataframe

In [23]:
for name in glob.glob(path+'/*.csv'):
    with open(name) as csvfile:
        df = pd.read_csv(name)
#         print(name)
#         readCSV = csv.reader(csvfile, delimiter=',')
#         for row in readCSV:
#             print(" ".join(row))

In [24]:
df

,ip,date,time,zone,cik,accession,extention,code,size,idx,norefer,noagent,find,crawler,browser
